# FateZ Multiomic Pertubation Effect Prediction(?)
This notebook demonstrate how to implement Pertubation Effect Prediction method with FateZ's modules.

In [2]:
import os
import sys
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import pandas as pd
import fatez.lib as lib
import fatez.test as test
import fatez.model as model
import fatez.tool.JSON as JSON
import fatez.process as process
import fatez.process.worker as worker
import fatez.process.fine_tuner as fine_tuner
import fatez.process.pre_trainer as pre_trainer
from pkg_resources import resource_filename

suppressor = process.Quiet_Mode()
device = [0]
dtype = torch.float32
worker.setup(device)
print('Done Init')

Done Init


### Build model and make some fake data first.

In [3]:
# Parameters
params = {
    'n_sample': 10,       # Fake samples to make
    'batch_size': 2,      # Batch size
}

# Load built-in config file
config = JSON.decode(resource_filename(
        __name__, '../../fatez/data/config/gat_bert_config.json'
    )
)

# Generate Fake data
faker = test.Faker(model_config = config, dtype = dtype, **params)
pertubation_dataloader = faker.make_data_loader()
result_dataloader = faker.make_data_loader()

# Make id of pertubation result the 'label' of each sample
for i,k in enumerate(pertubation_dataloader.dataset.samples):
    k.y = i
    
print('Done Fake Data')

Done Fake Data


### The model will be architecturally similar with a pretrainer

In [4]:
trainer = pre_trainer.Set(config, dtype = dtype, device=device)

print('Model Set')

Model Set


### However, the training part will be littel bit different
This part is modified based on pre_trainer.Trainer.train()

In [ ]:
report_batch = False
size = trainer.input_sizes

trainer.worker.train(True)
best_loss = 99
loss_all = 0
report = list()

for x,y in pertubation_dataloader:
    
    # Prepare input data as always
    input = [ele.to(trainer.device) for ele in x]
    
    # Mute some debug outputs
    suppressor.on()
    node_rec, adj_rec = trainer.worker(input)
    suppressor.off()
    
    # Prepare pertubation result data using a seperate dataloader
    y = [result_dataloader.dataset.samples[ele].to(trainer.device) for ele in y]
    # Please be noted here that this script is only reconstructing TF parts
    # To reconstruct whole genome, we can certainly add an additionaly layer which takes adj_rec and node_rec to do the job.
    node_results = torch.split(
        torch.stack([ele.x for ele in y], 0),
        node_rec.shape[1],
        dim = 1
    )[0]
    adj_results = lib.get_dense_adjs(
        y, (size['n_reg'],size['n_node'],size['edge_attr'])
    )
    
    # Get total loss
    loss = worker.criterion(node_rec, node_results)
    if adj_rec is not None:
        loss += worker.criterion(adj_rec, adj_results)
    
    # Some backward stuffs here
    loss.backward()
    nn.utils.clip_grad_norm_(worker.model.parameters(), worker.max_norm)
    worker.optimizer.step()
    worker.optimizer.zero_grad()

    # Accumulate
    best_loss = min(best_loss, loss.item())
    loss_all += loss.item()

    # Some logs
    if report_batch: report.append([loss.item()])


worker.scheduler.step()
report.append([loss_all / len(pertubation_dataloader)])
report = pd.DataFrame(report)
report.columns = ['Loss', ]
print(report)

### In the case of tuning unlabeled data, which does not have pertubation results... 
We shall set another trainer using previous model.

In [16]:
tuner = pre_trainer.Set(config, prev_model = trainer.model, dtype = dtype, device = device)

# Some new fake data
tuner_dataloader = faker.make_data_loader()

# And the tuning process is also based on input reconstruction as pretraining
suppressor.on()
report = tuner.train(tuner_dataloader, report_batch = False,)
suppressor.off()
print(report)

       Loss
0  2.158533


### Then we shall just use trainer object to make predictions.
Similar with the training block above for trainer, but no need to prepare y.

In [20]:
trainer.model.eval()

for x,_ in tuner_dataloader:
    
    # Prepare input data as always
    input = [ele.to(trainer.device) for ele in x]
    
    # Mute some debug outputs
    suppressor.on()
    node_rec, adj_rec = trainer.model(input)
    suppressor.off()
    print(node_rec, adj_rec)

tensor([[[ 0.5430, -0.1700],
         [ 0.3680, -0.3293],
         [ 0.0803, -0.9275],
         [ 0.2536, -0.8828]],

        [[ 0.2999, -0.3589],
         [ 0.1727, -0.4706],
         [ 0.0262, -0.8993],
         [ 0.3756, -0.8804]]], device='cuda:0', grad_fn=<ViewBackward0>) tensor([[[-5.2139e-02, -5.6997e-01, -4.4778e-01, -9.6441e-01, -1.1520e+00,
          -3.1230e-01, -7.3601e-01,  2.4879e-01, -1.4497e+00, -1.1241e+00],
         [-1.8447e-02, -4.3945e-01, -5.7454e-01, -1.0063e+00, -1.1827e+00,
          -4.1978e-01, -8.2745e-01,  1.2897e-01, -1.3348e+00, -1.2095e+00],
         [ 6.7265e-01,  7.5390e-01, -1.6032e+00, -3.6866e-02, -8.0894e-01,
           6.2469e-02, -4.4538e-01, -3.6848e-01,  1.0399e-03, -5.0661e-01],
         [ 2.9911e-01,  9.9707e-03, -1.3654e+00, -4.6959e-01, -1.3487e+00,
          -1.5249e-01, -5.8249e-01, -6.0127e-02, -8.8377e-01, -9.6960e-01]],

        [[-1.0271e-02, -3.8699e-01, -5.8424e-01, -1.0390e+00, -1.1630e+00,
          -4.6992e-01, -8.7301e-01,  8.17